In [1]:
import os
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"]="0"
igfold = 4
os.system('mkdir -p ./data_folds/fold_' + str(igfold))

0

In [2]:
!nvidia-smi

Sat Jan 14 08:53:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:86:00.0 Off |                    0 |
| N/A   36C    P0    44W / 163W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:89:00.0 Off |                    0 |
| N/A   

In [3]:
import cudf
import glob
import gc

from tqdm import tqdm

In [4]:
lb = False
path = '../data/'
type_weight = {0:1, 1:6, 2:3}
no_files = 1
path = '../../data/'
    
df_type = cudf.DataFrame({
    'type': ['clicks', 'carts', 'orders'],
    'type_': [0, 1, 2]
})

def list_in_chunks(files, no_chunks=10):
    out = [[] for _ in range(no_chunks)]
    for i, file in enumerate(files):
        out[i%no_chunks].append(file)
    return(out)

In [5]:
import pickle

sessions = pickle.load(open('./data/sessions_eval.pickle', 'rb'))
if igfold == 0:
    sess_eval = sessions[0]+sessions[1]
elif igfold == 1:
    sess_eval = sessions[2]+sessions[3]
elif igfold == 2:
    sess_eval = sessions[4]+sessions[5]
elif igfold == 3:
    sess_eval = sessions[6]+sessions[7]
elif igfold == 4:
    sess_eval = sessions[8]+sessions[9]
print(len(sess_eval))

356746


In [6]:
# !rm -rf ./data/
!mkdir -p ./data/

In [7]:
files = sorted(
    glob.glob(path + '/train/interim/*.parquet')
)
files_split = [glob.glob('../../data/test.parquet')] + [glob.glob('./data/xgb_train_x.parquet')] + list_in_chunks(files, no_chunks=len(files)//no_files)

In [8]:
len(files_split)

131

### 0) Clicks->Orders

In [9]:
!rm -r /tmp/data/tmp3
!mkdir -p /tmp/data/tmp3/split

In [10]:
out = []
for e, file in tqdm(enumerate(files_split)):
    df = cudf.read_parquet(file).reset_index(drop=True)
    df = df.merge(
        df_type,
        how='left',
        on='type'
    )
    if any([True for x in file if 'xgb_train_x.parquet' in x]):
        print(file)
        df = df[df['session'].isin(sess_eval)]
    else:
        df = df.loc[~(df['session'].isin(sess_eval))]

    df['session'] = df['session'].astype('int32')
    df['aid'] = df['aid'].astype('int32')
    df.ts = (df.ts/1000).astype('int32')
    df.drop(['type'], axis=1, inplace=True)
    df = df.rename(columns={'type_': 'type'})

    df = df.sort_values(['session','ts'],ascending=[True,True])
    df['k'] = np.arange(len(df))
    # USE TAIL OF SESSION
    #df = df.reset_index(drop=True)
    #df['n'] = df.groupby('session').cumcount()
    #df = df.loc[df.n<100].drop('n',axis=1)
    # CREATE PAIRS
    df = df.merge(df.drop_duplicates(['session','aid']),on=['session'])
    df = df.loc[ ((df.k_y - df.k_x).abs()>=1) & ((df.k_y - df.k_x).abs()<=2) & (df.aid_x != df.aid_y) ]
    # MEMORY MANAGEMENT COMPUTE IN PARTS
    #df = df.loc[(df.aid_x >= PART*SIZE)&(df.aid_x < (PART+1)*SIZE)]
    #df = df.sort_values('ts_x',ascending=False)
    # ASSIGN WEIGHTS
    df = df[['session', 'aid_x', 'aid_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
    df['wgt'] = 1 #df.type_y.map(type_weight)            
    #df['wgt'] = 1 + 3*(df.ts_x - 1659304800)/(1662328791-1659304800)

    df = df[['aid_x','aid_y','wgt']]
    df.wgt = df.wgt.astype('float32')
    df = df.groupby(['aid_x','aid_y']).wgt.sum()

    
    #out.append(df.reset_index().to_pandas())
    df.reset_index().to_parquet('/tmp/data/tmp3/split/split_' + str(e) + '.parquet')
    del df
    gc.collect()

1it [00:02,  2.74s/it]

['./data/xgb_train_x.parquet']


131it [03:38,  1.67s/it]


In [ ]:
files_split1 = glob.glob('/tmp/data/tmp3/split/split_*.parquet')
tmp = cudf.read_parquet(files_split1[0])
for iifile, file in enumerate(files_split1[1:]):
    print(file)
    df2 = cudf.read_parquet(file)
    tmp = cudf.concat([
        tmp,
        df2
    ])
    gc.collect()
    tmp = tmp.groupby(['aid_x','aid_y']).sum()
    tmp = tmp.reset_index()
    if ((iifile+1) % 50) == 0:
        print('Filter')
        print(tmp.shape)
        tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
        # SAVE TOP 40
        tmp = tmp.reset_index(drop=True)
        tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
        tmp = tmp.loc[tmp.n<150].drop('n',axis=1)
    del df2
    gc.collect()

/tmp/data/tmp3/split/split_59.parquet
/tmp/data/tmp3/split/split_111.parquet
/tmp/data/tmp3/split/split_9.parquet
/tmp/data/tmp3/split/split_36.parquet
/tmp/data/tmp3/split/split_39.parquet
/tmp/data/tmp3/split/split_69.parquet
/tmp/data/tmp3/split/split_60.parquet
/tmp/data/tmp3/split/split_25.parquet
/tmp/data/tmp3/split/split_61.parquet
/tmp/data/tmp3/split/split_127.parquet
/tmp/data/tmp3/split/split_104.parquet
/tmp/data/tmp3/split/split_110.parquet
/tmp/data/tmp3/split/split_73.parquet
/tmp/data/tmp3/split/split_81.parquet
/tmp/data/tmp3/split/split_24.parquet
/tmp/data/tmp3/split/split_6.parquet
/tmp/data/tmp3/split/split_105.parquet
/tmp/data/tmp3/split/split_28.parquet
/tmp/data/tmp3/split/split_91.parquet
/tmp/data/tmp3/split/split_107.parquet
/tmp/data/tmp3/split/split_74.parquet
/tmp/data/tmp3/split/split_35.parquet
/tmp/data/tmp3/split/split_45.parquet
/tmp/data/tmp3/split/split_65.parquet
/tmp/data/tmp3/split/split_58.parquet
/tmp/data/tmp3/split/split_11.parquet
/tmp/dat

In [ ]:
gc.collect()

tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
# SAVE TOP 40
tmp = tmp.reset_index(drop=True)
tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
tmp = tmp.loc[tmp.n<80].drop('n',axis=1)

tmp.to_parquet('./data_folds/fold_' + str(igfold) + '/top_40_aids_v220.parquet')

In [ ]:
del tmp
gc.collect()